# Trabajo Práctico 1 
Hofkamp Nataly

In [2]:
pip install torch

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.7 MB/s eta 0:00:00:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 9.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.8 MB/s eta 0:00:00:00:010:03m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 644.3 kB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 9.1 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.3/944.3 kB 5.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
import torch
import numpy as np
import torch.nn as nn
from typing import List
import gymnasium as gym
import torch.optim as optim
from gymnasium import spaces
from torch.distributions import Categorical


# 1.Entornos de Prueba

## a. TwoAZeroObsOneStep



In [11]:


class TwoAZeroObsOneStep(gym.Env):
    def __init__(self):
        super().__init__()
        self.observation_space = spaces.Discrete(1)  # siempre 0
        self.action_space = spaces.Discrete(2)       # dos acciones: 0 o 1
        self.correct_action = 0                      # definimos cuál es la "buena"

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        observation = 0
        info = {}
        return observation, info

    def step(self, action):
        reward = 1 if action == self.correct_action  else -1
        observation = 0
        terminated = True       # solo un paso
        truncated = False
        info = {}
        return observation, reward, terminated, truncated, info


testeo

In [18]:
env = TwoAZeroObsOneStep()
obs, _ = env.reset() 
terminated = False
truncated  = False
while not terminated and not truncated:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Acción: {action}, Reward: {reward}, Observation:{obs}")

Acción: 1, Reward: -1, Observation:0


## b.TwoARandomObsOneStep


In [19]:

class TwoARandomObsOneStep(gym.Env):
    def __init__(self):
        super().__init__()
        self.observation_space = spaces.Discrete(2)  # 0 = S1, 1 = S2
        self.action_space = spaces.Discrete(2)       # dos acciones: 0 o 1
        self.state = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # elegir S1 (0) o S2 (1) aleatoriamente
        self.state = np.random.choice([0, 1])
        info = {}
        return self.state, info

    def step(self, action):
        # Recompensa depende del estado
        if self.state == 0:   # S1
            reward = 1 if action == 0 else -1
        else:                 # S2
            reward = 1 if action == 1 else -1

        observation = self.state
        terminated = True    # un solo paso
        truncated = False
        info = {}

        return observation, reward, terminated, truncated, info


In [20]:
env = TwoARandomObsOneStep()
obs, _ = env.reset() 
terminated = False
truncated  = False
while not terminated and not truncated:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Acción: {action}, Reward: {reward}, Observation:{obs}")

Acción: 0, Reward: 1, Observation:0


## c. LineWorldEasyEnv

In [21]:
class LineWorldEasyEnv(gym.Env):
    def __init__(self):
        super().__init__()
        # Observaciones = posiciones de 1 a 6
        self.observation_space = spaces.Discrete(6)  
        # Acciones = izquierda (0) o derecha (1)
        self.action_space = spaces.Discrete(2)       
        self.state = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = 1  # posición inicial
        info = {}
        return self.state, info

    def step(self, action):
        # Mover
        if action == 0:   # izquierda
            if self.state > 1:
                self.state -= 1
        elif action == 1: # derecha
            if self.state < 6:
                self.state += 1

        # Recompensa
        reward = 1 if self.state == 6 else 0

        # Condición de fin
        terminated = (self.state == 6)
        truncated = False
        info = {}

        return self.state, reward, terminated, truncated, info


In [22]:
env = LineWorldEasyEnv()
obs, _ = env.reset() 
terminated = False
truncated  = False
while not terminated and not truncated:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Acción: {action}, Reward: {reward}, Observation:{obs}")

Acción: 0, Reward: 0, Observation:1
Acción: 1, Reward: 0, Observation:2
Acción: 0, Reward: 0, Observation:1
Acción: 0, Reward: 0, Observation:1
Acción: 1, Reward: 0, Observation:2
Acción: 1, Reward: 0, Observation:3
Acción: 1, Reward: 0, Observation:4
Acción: 1, Reward: 0, Observation:5
Acción: 1, Reward: 1, Observation:6


## d.LineWorldMirrorEnv

In [23]:

class LineWorldMirrorEnv(gym.Env):
    def __init__(self):
        super().__init__()
        self.observation_space = spaces.Discrete(4)  # estados 1 a 4
        self.action_space = spaces.Discrete(2)       # 0 = izquierda, 1 = derecha
        self.state = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = 1  # posición inicial
        info = {}
        return self.state, info

    def step(self, action):
        # Invertir acciones en el estado 2
        if self.state == 2:
            action = 1 - action

        # Movimiento normal
        if action == 0 and self.state > 1:   # izquierda
            self.state -= 1
        elif action == 1 and self.state < 4: # derecha
            self.state += 1

        reward = -1

        # Condición de fin
        terminated = (self.state == 4)  # objetivo
        truncated = False
        info = {}

        return self.state, reward, terminated, truncated, info


In [25]:
env = LineWorldMirrorEnv()
obs, _ = env.reset() 
terminated = False
truncated  = False
while not terminated and not truncated:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Acción: {action}, Reward: {reward}, Observation:{obs}")

Acción: 1, Reward: -1, Observation:2
Acción: 1, Reward: -1, Observation:1
Acción: 1, Reward: -1, Observation:2
Acción: 1, Reward: -1, Observation:1
Acción: 1, Reward: -1, Observation:2
Acción: 1, Reward: -1, Observation:1
Acción: 0, Reward: -1, Observation:1
Acción: 1, Reward: -1, Observation:2
Acción: 0, Reward: -1, Observation:3
Acción: 0, Reward: -1, Observation:2
Acción: 1, Reward: -1, Observation:1
Acción: 1, Reward: -1, Observation:2
Acción: 0, Reward: -1, Observation:3
Acción: 1, Reward: -1, Observation:4


## 2. Implementación del Algoritmo REINFORCE

### política 

In [ ]:
class Policy(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 20),
            nn.ELU(),
            nn.Linear(20, 20),
            nn.ELU(),
            nn.Linear(20, output_dim)  # logits crudos
        )
    
    def forward(self, x):
        return self.net(x)   # devuelve logits

def get_policy(obs, policy):
    logits = policy(obs)
    return Categorical(logits=logits)

### rewards

In [ ]:

def compute_returns(rewards, gamma=0.99):
    """ Computa los retornos descontados """
    G, returns = 0, []
    for r in reversed(rewards):
        G = r + gamma * G
        returns.insert(0, G)
    return torch.tensor(returns, dtype=torch.float32)


### La función de pérdida REINFORCE $\mathcal{L}(\theta)$

In [ ]:

def compute_loss(policy, obs, acts, returns):
    """Computa la pérdida del gradiente de política."""
    dist = get_policy(obs, policy)
    log_probs = dist.log_prob(acts)
    return -(log_probs * returns).sum()

def reinforce(env, policy, optimizer, batch_size=10, gamma=0.99):
    all_rewards, batch_obs,batch_acts, batch_returns  = [],[],[],[]
    
    for episode in range(batch_size):
        state = env.reset()
        rewards,states,actions= [],[],[]
        
        done = False
        while not done:
            state_t = torch.tensor(state, dtype=torch.float32)
            dist = get_policy(state_t, policy)
            action = dist.sample()
            
            state, reward, done, _ = env.step(action.item())
            
            states.append(state_t)
            actions.append(action)
            rewards.append(reward)
        
        # calcular retornos descontados
        discounted = compute_returns(rewards, gamma)
        
        batch_obs.extend(states)
        batch_acts.extend(actions)
        batch_returns.extend(discounted)
        all_rewards.append(sum(rewards))
    
    # tensores
    batch_obs = torch.stack(batch_obs)
    batch_acts = torch.tensor(batch_acts)
    batch_returns = torch.tensor(batch_returns, dtype=torch.float32)
    
    # normalizar retornos
    batch_returns = (batch_returns - batch_returns.mean()) / (batch_returns.std() + 1e-8)

    # pérdida vía compute_loss
    loss = compute_loss(policy, batch_obs, batch_acts, batch_returns)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return all_rewards
